In [1]:
import os
import warnings
warnings.simplefilter('ignore')

import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
census_tract = gpd.read_file('https://cities-socio-economic-vulnerability.s3.eu-west-3.amazonaws.com/case-studies/campinas/census_trac_geo.geojson')

In [3]:
census_tract.dropna(how='all', axis=1, inplace=True)

In [4]:
census_tract = census_tract.rename({'p_5or6':'p_5or6_m'}, axis = 1)

In [5]:
male = ['p_5or6_m', 'p_7to9_m', 'p_10to14_m', 'p_15to19_m', 'p_20to24_m', 'p_25to29_m', 'p_30to34_m', 'p_35to39_m', 'p_40to44_m', 'p_45to49_m', 'p_50to54_m', 'p_55to59_m', 'p_60to69_m', 'p_70m_m']

In [6]:
census_tract['p_tot_m'] = 0

for i in male:
    census_tract['p_tot_m'] = census_tract['p_tot_m'] + census_tract[i]

census_tract['p_tot_f'] = census_tract['p_tot'] - census_tract['p_tot_m']

In [7]:
census_tract['p_0to14yr'] = census_tract['p_0to4'] + census_tract['p_5to9'] + census_tract['p_10to14']
census_tract['p_15to19yr'] = census_tract['p_15to19']
census_tract['p_20to59yr'] = census_tract['p_20to24'] + census_tract['p_25to29'] + census_tract['p_30to34'] + census_tract['p_35to39'] + census_tract['p_40to44'] + census_tract['p_45to49'] + census_tract['p_50to54'] + census_tract['p_55to59']
census_tract['p_60+yr'] = census_tract['p_60to69'] + census_tract['p_70m']

In [8]:
census_tract['p_10+yr'] = census_tract['p_10to14'] + census_tract['p_15to19yr'] + census_tract['p_20to59yr'] + census_tract['p_60+yr']

In [9]:
census_tract['p_mean_inc1_city'] = (census_tract['p_mean_inc1']*census_tract['p_10+yr']).sum()/census_tract['p_10+yr'].sum()

In [10]:
var = ['p_tot', 'p_tot_f', 'p_tot_m'] + ['p_wht', 'p_black', 'p_yellow', 'p_brown', 'p_indig'] + ['p_0to14yr', 'p_15to19yr', 'p_20to59yr', 'p_60+yr'] + ['p_mean_inc1', 'p_mean_inc1_city']

In [11]:
keep = ['code_tract'] + var + ['geometry']

In [12]:
census_tract_keep = census_tract[keep]

In [13]:
for k in var[1:-2]:
    census_tract_keep['prop1_'+k] = round(100*census_tract_keep[k]/census_tract_keep['p_tot'],4)

In [14]:
for k in var[:-2]:
    census_tract_keep['prop2_'+k] = round(100*census_tract_keep[k]/census_tract_keep[k].sum(),4)

In [15]:
census_tract_keep['prop2_' + 'p_mean_inc1'] = round(100*census_tract_keep['p_mean_inc1']/census_tract_keep['p_mean_inc1_city'],4)

In [18]:
census_tract_keep['p_10+yr'] = census_tract['p_10+yr']

In [21]:
census_tract_keep.to_file('../Data/census_tract_keep.geojson')